In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("spam.csv")
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.Category.value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [5]:
df['spam'] = df['Category'].apply(lambda x: 1 if x =='spam' else 0)

In [7]:
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.Message, df.spam, test_size=0.2)

In [12]:
X_train[:4]

C:\Users\User\AppData\Local\Temp\ipykernel_6128\2167669864.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  X_train[:4]


4102    GSOH? Good with SPAM the ladies?U could b a ma...
105          Umma my life and vava umma love you lot dear
3470    All day working day:)except saturday and sunday..
1602    Carlos is taking his sweet time as usual so le...
Name: Message, dtype: object

# Creating Bag Of Words

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

v = CountVectorizer()

X_train_cv = v.fit_transform(X_train.values)
X_train_cv

<4457x7667 sparse matrix of type '<class 'numpy.int64'>'
	with 58667 stored elements in Compressed Sparse Row format>

In [14]:
X_train_cv.toarray()[:2][0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [15]:
X_train_cv.shape

(4457, 7667)

In [18]:
X_train_np = X_train_cv.toarray()
X_train_np[1]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [19]:
np.where(X_train_np[0]!=0)

(array([ 113,  398,  528, 1829, 1994, 2748, 3098, 3142, 3207, 3213, 3787,
        3940, 4278, 4385, 4465, 4556, 4868, 5640, 6274, 6412, 6738, 7055,
        7469], dtype=int64),)

# Training By Naive Bayes

In [20]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train_cv, y_train)

MultinomialNB()

In [21]:
X_test_cv = v.transform(X_test)

In [22]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_cv)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       954
           1       0.99      0.93      0.96       161

    accuracy                           0.99      1115
   macro avg       0.99      0.96      0.98      1115
weighted avg       0.99      0.99      0.99      1115



In [24]:
emails = [
    #'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]

emails_count = v.transform(emails)
model.predict(emails_count)

array([1], dtype=int64)

# Training By SkLearn Pipeline

In [25]:
from sklearn.pipeline import Pipeline

sv = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('nb', MultinomialNB())
])

In [26]:
sv.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

In [27]:
from sklearn.metrics import classification_report

y_pred = sv.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       954
           1       0.99      0.93      0.96       161

    accuracy                           0.99      1115
   macro avg       0.99      0.96      0.98      1115
weighted avg       0.99      0.99      0.99      1115



In [29]:
emails = [
    #'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]

sv.predict(emails)

array([1], dtype=int64)